In [1]:
from rdflib import Graph, Literal, RDF, URIRef, Namespace
import pandas as pd
import requests
import json
import pyconll

In [2]:
def generate_expression_id(expression):
    expression = expression.replace(",", "").replace("?", "")
    w = expression.split(" ")
    return "_".join(w[ : 4])

### Latin philosophical expression

In [3]:
df_latin = pd.read_csv('./Latin-Philosophical-Expressions.csv', encoding = 'utf-8')

In [4]:
df_latin = df_latin.fillna('')

In [5]:
df_latin.head()

,id,latin_expression,expression_url,branch,branch_url,concept,translation_eng,sense_eng,translation_ita,sense_ita,sense_lat
0,a_dicto_secundum_quid,a dicto secundum quid ad dictum simpliciter,https://www.wikidata.org/wiki/Q2456273,logic,https://www.wikidata.org/wiki/Q8078,The phrase 'a dicto secundum quid ad dictum si...,from a statement qualified to a statement unqu...,This phrase refers to the logical fallacy of m...,dal detto secondo il quale al detto semplicemente,Questo concetto filosofico si riferisce alla d...,A dicto secundum quid ad dictum simpliciter si...
1,a_dicto_simpliciter_ad,a dicto simpliciter ad dictum secundum quid,https://www.wikidata.org/wiki/Q4660909,logic,https://www.wikidata.org/wiki/Q8078,The Latin phrase 'a dicto simpliciter ad dictu...,from the saying absolutely to the saying with ...,This phrase refers to the logical fallacy of m...,da un detto semplice a un detto secondo quanto,Questo concetto filosofico indica la distinzio...,A dicto simpliciter ad dictum secundum quid si...
2,a_fortiori,a fortiori,https://www.wikidata.org/wiki/Q1753631,logic,https://www.wikidata.org/wiki/Q8078,The Latin phrase 'a fortiori' translates to 'f...,a fortiori,"A Latin term meaning 'with stronger reason', u...",a maggior ragione,Il termine 'a fortiori' indica un ragionamento...,Argumentum a fortiori: si aliquid verum est in...
3,a_necesse_ad_esse,a necesse ad esse valet consequentia,,logic,https://www.wikidata.org/wiki/Q8078,The Latin phrase 'a necesse ad esse valet cons...,"From necessity to being, the consequence holds.",This phrase suggests that if something is nece...,Da necessario a essere vale la conseguenza,Questo concetto filosofico indica che se qualc...,"Consequentia ad necessarium esse valet, si res..."
4,a_posteriori,a posteriori,https://www.wikidata.org/wiki/Q300637,epistemology,https://www.wikidata.org/wiki/Q9471,The term 'a posteriori' is a Latin phrase mean...,from the latter,A method of reasoning or knowledge that is der...,a posteriori,Il termine 'a posteriori' si riferisce a una c...,Cognitio ex experientia; veritas quae ex obser...


### Different interpretations (senses)

In [6]:
df_senses = pd.read_csv('./Latin-Expressions-Senses.csv', encoding = 'utf-8')

In [7]:
df_senses = df_senses.fillna('')

In [8]:
df_senses.head()

,expression,associated_with,sense,id,associated_with_url
0,a dicto secundum quid ad dictum simpliciter,Aristotle,Aristotle uses this expression to differentiat...,a_dicto_secundum_quid,https://www.wikidata.org/wiki/Q84473023
1,a dicto secundum quid ad dictum simpliciter,Thomas Aquinas,Aquinas interprets this phrase in the context ...,a_dicto_secundum_quid,https://www.wikidata.org/wiki/Q9438
2,a dicto secundum quid ad dictum simpliciter,Immanuel Kant,Kant might interpret this expression in relati...,a_dicto_secundum_quid,https://www.wikidata.org/wiki/Q9312
3,a dicto secundum quid ad dictum simpliciter,Leibniz,Leibniz could view this distinction as a refle...,a_dicto_secundum_quid,https://www.wikidata.org/wiki/Q9047
4,a dicto simpliciter ad dictum secundum quid,Aristotle,Aristotle uses this expression to differentiat...,a_dicto_simpliciter_ad,https://www.wikidata.org/wiki/Q84473023


## Linking to LiLa lemmas

In [9]:
with open("./lila_linking_final.json", "r") as f:
    text = f.read()
lila_linking = json.loads(text)

## Linked concepts

In [10]:
linked_concepts = [
    ["a dicto secundum quid ad dictum simpliciter","a dicto simpliciter ad dictum secundum quid"],
    ["a priori","a posteriori","a fortiori"],
    ["argumentum ad baculum","argumentum ad hominem","argumentum ad populum"],
    ["bellum omnium contra omnes","homo homini lupus"],
    ["modus ponens","modus tollens"],
    ["de dicto","de facto","de jure","de re"],
    ["in actu","in esse","in potentia"],
    ["mundus intelligibilis","mundus sensibilis"],
    ["per accidens","per se"],
    ["res cogitans", "res extensa"],
    ["tabula rasa","nihil in intellectu nisi prius in sensu"]
]

In [11]:
linked_concepts_list = [generate_expression_id(concept)  for concepts in linked_concepts for concept in concepts]

In [12]:
linked_concepts_dict = {c : {'uri':'', 'links' : []} for c in set(linked_concepts_list)}

In [13]:
for concepts in linked_concepts:
    for concept in concepts:
        concept_id = generate_expression_id(concept)       
        for linked_concept in concepts:
            if linked_concept != concept:
                 linked_concepts_dict[concept_id]['links'].append(generate_expression_id(linked_concept))


In [14]:
linked_concepts_dict

{'res_cogitans': {'uri': '', 'links': ['res_extensa']},
 'mundus_sensibilis': {'uri': '', 'links': ['mundus_intelligibilis']},
 'de_dicto': {'uri': '', 'links': ['de_facto', 'de_jure', 'de_re']},
 'argumentum_ad_baculum': {'uri': '',
  'links': ['argumentum_ad_hominem', 'argumentum_ad_populum']},
 'in_actu': {'uri': '', 'links': ['in_esse', 'in_potentia']},
 'de_re': {'uri': '', 'links': ['de_dicto', 'de_facto', 'de_jure']},
 'homo_homini_lupus': {'uri': '', 'links': ['bellum_omnium_contra_omnes']},
 'modus_tollens': {'uri': '', 'links': ['modus_ponens']},
 'res_extensa': {'uri': '', 'links': ['res_cogitans']},
 'modus_ponens': {'uri': '', 'links': ['modus_tollens']},
 'a_posteriori': {'uri': '', 'links': ['a_priori', 'a_fortiori']},
 'argumentum_ad_hominem': {'uri': '',
  'links': ['argumentum_ad_baculum', 'argumentum_ad_populum']},
 'nihil_in_intellectu_nisi': {'uri': '', 'links': ['tabula_rasa']},
 'argumentum_ad_populum': {'uri': '',
  'links': ['argumentum_ad_baculum', 'argumentum

## UD Pipe

In [15]:
def call_udpipe_and_parse(text, model='latin-evalatin24-240520'):
    url = 'https://lindat.mff.cuni.cz/services/udpipe/api/process'
    params = {
        'tokenizer': '',
        'tagger': '',
        'parser': '',
        'data': text,
        'model': model
    }

    response = requests.post(url, data=params)
    
    if response.status_code == 200:
        result = response.json()
        conllu_result = result.get('result')
        if conllu_result:
            # Parse CoNLL-U with pyconll
            conll_data = pyconll.load_from_string(conllu_result)
            return conll_data
        else:
            print("No CoNLL-U output found. Error:", result.get('error', 'Unknown error'))
            return None
    else:
        print(f"HTTP Error {response.status_code}: {response.text}")
        return None

## Create triples

In [16]:
def create_uri(namespace, resource_type, identifier, language=None):
   
    identifier = str(identifier).lower().replace(' ', '-').replace('_', '-')
    
    if language:
        # Language-specific resources
        return namespace[f"{resource_type}/{language}/{identifier}"]
    else:
        # Language-neutral resources
        return namespace[f"{resource_type}/{identifier}"]


In [43]:
from rdflib import Graph, URIRef, Literal, Namespace
from rdflib.namespace import RDF, RDFS, OWL, DC, FOAF, SKOS, DCTERMS

# Define namespaces
ONTOLEX = Namespace("http://www.w3.org/ns/lemon/ontolex#")
LEXINFO = Namespace("http://www.lexinfo.net/ontology/2.0/lexinfo#")
LIME = Namespace("http://www.w3.org/ns/lemon/lime#")
VARTRANS = Namespace("http://www.w3.org/ns/lemon/vartrans#")
PROV = Namespace("http://www.w3.org/ns/prov#")
WD = Namespace("http://www.wikidata.org/entity/")
DCT = Namespace("http://purl.org/dc/terms/")
OA = Namespace("http://www.w3.org/ns/oa#")

POWLA = Namespace("http://purl.org/powla/powla.owl#")
NIF = Namespace("http://persistence.uni-leipzig.org/nlp2rdf/ontologies/nif-core#")
LILA = Namespace("http://lila-erc.eu/ontologies/lila/")

# My namespace
LP = Namespace("http://latinphilosophy.org/")

In [62]:
g = Graph()
    
# Binding
g.bind("ontolex", ONTOLEX)
g.bind("lexinfo", LEXINFO)
g.bind("lime", LIME)
g.bind("skos", SKOS)
g.bind("dcterms", DCTERMS)
g.bind("vartrans", VARTRANS)
g.bind("prov", PROV)
g.bind("dct", DCT)
g.bind("powla", POWLA)
g.bind("nif", NIF)
g.bind("foaf", FOAF)
g.bind("lila", LILA)
g.bind("oa", OA)

g.bind("lp", LP)

# Corpus
corpus_uri =LP["corpus"]
g.add((corpus_uri, RDF.type, POWLA.Corpus))
g.add((corpus_uri, RDFS.label, Literal('Philosophical Latin Expressions Corpus')))  
g.add((corpus_uri, DCT.creator, Literal('Giovanna Tonazzo')))  

# Document and citation layers
doc_uri = create_uri(LP, "layer", "document")
g.add((doc_uri, RDF.type, POWLA.DocumentLayer))
g.add((doc_uri, RDFS.label, Literal('Document Layer')))
g.add((doc_uri, POWLA.hasDocument, corpus_uri))

cit_uri = create_uri(LP, "layer", "expression")
g.add((cit_uri, RDF.type, POWLA.DocumentLayer))
g.add((cit_uri, RDFS.label, Literal('Expression Layer')))
g.add((cit_uri, POWLA.hasDocument, corpus_uri))

# Create annotation layers
ud_layer = create_uri(LP, "layer", "ud")
g.add((ud_layer, RDF.type, POWLA.Layer))
g.add((ud_layer, RDFS.label, Literal('UD Annotation Layer')))  
                                             
sem_layer = create_uri(LP, "layer", "semantics")
g.add((sem_layer, RDF.type, POWLA.Layer))
g.add((sem_layer, RDFS.label, Literal('Semantic Annotation Layer')))  
         
dep_layer = create_uri(LP, "layer", "dependency")
g.add((dep_layer, RDF.type, POWLA.Layer))
g.add((dep_layer, RDFS.label, Literal('Dependency Annotation Layer')))

g.add((corpus_uri, POWLA.hasLayer, ud_layer))
g.add((corpus_uri, POWLA.hasLayer, sem_layer))
g.add((corpus_uri, POWLA.hasLayer, dep_layer))

# Lexicon for Latin and English
lat_lex_uri = create_uri(LP, "lexicon", "la")
g.add((lat_lex_uri, RDF.type, LIME.lexicon))
g.add((lat_lex_uri, RDFS.label, Literal('Latin lexicon')))

eng_lex_uri = create_uri(LP, "lexicon", "en")
g.add((eng_lex_uri, RDF.type, LIME.lexicon))
g.add((eng_lex_uri, RDFS.label, Literal('English lexicon')))

# Create  triples for each expression
for index, row in df_latin.iterrows():
    
    expr_id = row['id']
    latin_expr = row['latin_expression']    
    branch = row['branch']
    branch_url = row['branch_url']
    expr_url = row['expression_url']        
    concept = row['concept']
    expr_trans_eng = row['translation_eng']
    expr_sense_eng = row['sense_eng']
    expr_sense_lat = row['sense_lat']

    print(latin_expr)
    #g.add((ONTOLEX.Form, RDFS.label, Literal('Form')))
    #g.add((ONTOLEX.LexicalEntry, RDFS.label, Literal('Lexical entry')))  
    #g.add((VARTRANS.Translation, RDFS.label, Literal('Translation')))
    #g.add((ONTOLEX.LexicalSense, RDFS.label, Literal('Lexical Sense')))
    #g.add((ONTOLEX.LexicalConcept, RDFS.label, Literal('Lexical Concept')))      

    # Concept
    concept_uri = create_uri(LP, "concept", index)
    g.add((concept_uri, RDF.type, SKOS.Concept))
    g.add((concept_uri, RDFS.label,  Literal(f'Concept of {latin_expr}', lang="en")))
    g.add((concept_uri, SKOS.definition, Literal(concept, lang="en"))) 
    sem_annot_uri = create_uri(LP, 'semantics', f'expr-{index}')
    g.add((sem_annot_uri, RDFS.label, Literal(f'Semantic annotation of {latin_expr}')))
    g.add((sem_annot_uri, RDF.type, POWLA.Node))
    g.add((sem_annot_uri, POWLA.hasLayer, sem_layer))
    g.add((sem_annot_uri, DC.subject, concept_uri))

    if linked_concepts_dict.get(expr_id):
        linked_concepts_dict[expr_id]['uri'] = concept_uri
    
    # Link to branch of philosophy
    branch_id = branch.lower().replace(" ", "-")
    branch_uri = create_uri(LP, "branch", branch_id)
    g.add((branch_uri, RDF.type, SKOS.Concept))
    g.add((branch_uri, SKOS.prefLabel, Literal(branch.capitalize(), lang="en")))
    g.add((concept_uri, SKOS.broader, branch_uri))   
    if branch_url:   
        g.add((branch_uri, RDFS.seeAlso, URIRef(branch_url)))
        
    # Link to Wikidata if available
    wiki_id = expr_url.split('/')[-1] if expr_url else None    
    if wiki_id:
        g.add((concept_uri, RDFS.seeAlso, WD[wiki_id]))
        
    # Lexical entry for Latin expression
    latin_entry_uri = create_uri(LP, "expression", index, "la")
    g.add((latin_entry_uri, RDF.type, ONTOLEX.LexicalEntry))
    g.add((latin_entry_uri, RDFS.label, Literal(latin_expr, lang="la")))    
    g.add((latin_entry_uri, RDF.type, POWLA.root))

    # Lexical form
    form_uri = create_uri(LP, "form", index, "la")
    g.add((form_uri, RDF.type, ONTOLEX.Form))
    g.add((latin_entry_uri, ONTOLEX.canonicalForm, form_uri))
    g.add((form_uri, ONTOLEX.writtenRep, Literal(latin_expr.capitalize(), lang="la")))

    # Lexical sense
    latin_sense_uri = create_uri(LP, "sense", index, "la")
    g.add((latin_sense_uri, RDF.type, ONTOLEX.LexicalSense))
    g.add((latin_entry_uri, ONTOLEX.sense, latin_sense_uri))    
    g.add((latin_sense_uri, RDFS.label, Literal(f'Latin sense of {latin_expr}', lang="la")))
    g.add((latin_sense_uri, SKOS.definition, Literal(expr_sense_lat, lang="la")))
    g.add((latin_sense_uri, ONTOLEX.reference, concept_uri))
    
    # Lexical entry for English translation
    english_entry_uri = create_uri(LP, "expression", index, "en")
    g.add((english_entry_uri, RDF.type, ONTOLEX.LexicalEntry))
    g.add((english_entry_uri, RDFS.label, Literal(expr_trans_eng, lang="en")))

    # Canonical form for English
    english_form_uri = create_uri(LP, "form", index, "en")
    g.add((english_form_uri, RDF.type, ONTOLEX.Form))
    g.add((english_entry_uri, ONTOLEX.canonicalForm, english_form_uri))
    g.add((english_form_uri, ONTOLEX.writtenRep, Literal(expr_trans_eng.capitalize(), lang="en")))

     # Sense for English translation
    english_sense_uri = create_uri(LP, "sense", index, "en")
    g.add((english_sense_uri, RDF.type, ONTOLEX.LexicalSense))
    g.add((english_entry_uri, ONTOLEX.sense, english_sense_uri))
    g.add((english_sense_uri, RDFS.label, Literal(f'English sense of {latin_expr}', lang="en")))
    g.add((english_sense_uri, SKOS.definition, Literal(expr_sense_eng, lang="en")))   
    g.add((english_sense_uri, ONTOLEX.reference, concept_uri ))
   
    
    #Translation relation
    trans_uri = create_uri(LP, "translation", index)
    g.add((trans_uri, RDF.type, VARTRANS.Translation))
    g.add((trans_uri, RDFS.label, Literal(f'Translation of {latin_expr}')))
    
    g.add((trans_uri, VARTRANS.source, latin_sense_uri))
    g.add((trans_uri, VARTRANS.target, english_sense_uri))
   
    # Different interpretations
    for sense_id , sense in df_senses[df_senses['id'] == expr_id].iterrows(): 
         # Lexical sense
        auth_id = sense['associated_with'].lower().replace(" ", "-")
        sense_uri = create_uri(LP, "sense", f'expr-{index}/{auth_id}', "en")
        g.add((sense_uri, RDF.type, ONTOLEX.concept))   
        g.add((sense_uri, RDFS.label, Literal(f'{sense['associated_with']} sense of {latin_expr}', lang="en")))
        g.add((sense_uri, SKOS.definition, Literal(sense['sense'], lang="en")))
        g.add((concept_uri, SKOS.narrower, sense_uri))

        auth_uri = create_uri(LP, "creator", auth_id)
        g.add((auth_uri,  RDFS.label, Literal(sense['associated_with'], lang="en")))
        g.add((auth_uri, FOAF.page, URIRef(sense['associated_with_url'])))
        g.add((sense_uri, DCT.creator, auth_uri))   
        g.add((concept_uri, DCT.creator, auth_uri))
        if sense['associated_with_url'] != '':   
            g.add((auth_uri, RDFS.seeAlso, URIRef(sense['associated_with_url'])))       
        
        if auth_id == 'plato':
            plato_uri = auth_uri
            
    # Add to citation layer
    g.add((latin_entry_uri, POWLA.hasLayer, cit_uri))

    # Add to lexicon layer
    g.add((latin_entry_uri, LIME.entry, lat_lex_uri))
    g.add((english_entry_uri, LIME.entry, eng_lex_uri))
    
    # Parse with UDPipe
    conll = call_udpipe_and_parse(latin_expr.replace(",","").replace("?",""))    
    if conll:                              
        for i, token in enumerate(conll[0]):
            
            # Terminal for the token
            token_id = i + 1
            powla_token_uri = create_uri(LP, 'token', f'expr-{index}/{token_id}') 
            token_form = token.form
                                                    
            g.add((powla_token_uri, RDF.type, POWLA.Terminal))
            g.add((powla_token_uri, RDFS.label, Literal(token_form, lang="la")))
            g.add((latin_entry_uri, POWLA.hasTerminal, powla_token_uri))
            g.add((powla_token_uri, POWLA.hasLayer, doc_uri))
    
            # First, next, last token relations
            if token_id == 1:
                g.add((latin_entry_uri, POWLA.first, powla_token_uri))        
            
            if token_id == len(conll[0]):
                g.add((latin_entry_uri, POWLA.last, powla_token_uri))
    
            if token_id > 1:
                prev_token_uri = create_uri(LP, 'token', f'expr_{index}/{token_id - 1}')
                g.add((powla_token_uri, POWLA.prev, prev_token_uri))
                g.add((prev_token_uri, POWLA.next, powla_token_uri))
    
            # Dependency relation
            if token.head:
                token_head = int(token.head)
                if token_head != 0:
                    dep_relation_uri = create_uri(LP, 'deprel', f'expr-{index}/{token_head}_{token_id}')
                    powla_head_uri = create_uri(LP, 'token', f'expr-{index}/{token_head}')      
                    g.add((dep_relation_uri, RDF.type, POWLA.Relation))
                    g.add((dep_relation_uri, RDFS.label, Literal(f'deprel: {token.deprel}')))
                    g.add((dep_relation_uri, POWLA.hasLayer, dep_layer))
                    g.add((dep_relation_uri, POWLA.hasSource, powla_head_uri))
                    g.add((dep_relation_uri, POWLA.hasTarget, powla_token_uri))
                    g.add((dep_relation_uri, POWLA.hasLabel, Literal(token.deprel)))    
            
            
            # UD annotations from conllu
            ud_node_uri = create_uri(LP, 'ud', f'expr-{index}/{token_id}')
            g.add((ud_node_uri, RDFS.label, Literal(f'UD annotation of {token_form}')))
            g.add((ud_node_uri, RDF.type, OA.annotation))
            g.add((ud_node_uri, POWLA.hasLayer, ud_layer))
            g.add((ud_node_uri, OA.hasTarget, powla_token_uri))
            #pos_node_uri = URIRef(f'https://universaldependencies.org/la/{token.upos}')
            pos_node_uri = create_uri(LP, 'pos', f'{token.upos}')
            g.add((pos_node_uri, RDFS.label, Literal(f'POS={token.upos}')))
            g.add((ud_node_uri, OA.hasBody, pos_node_uri))
    
            # Feats from conllu
            feats = dict(token.feats)
            for feat in feats:
                for f in feats[feat]:
                    #feat_node_uri = URIRef(f'https://universaldependencies.org/la/feat/{feat.capitalize()}#{f.capitalize()}')
                    feat_node_uri= create_uri(LP, 'feat', f'{feat.capitalize()}={f.capitalize()}')
                    g.add((feat_node_uri, RDFS.label, Literal(f'{feat.capitalize()}={f.capitalize()}')))
                    g.add((feat_node_uri, RDF.type, URIRef(f'https://universaldependencies.org/u/feat/{feat.capitalize()}#{f.capitalize()}')))
                    g.add((URIRef(f'https://universaldependencies.org/la/feat/{feat.capitalize()}#{f.capitalize()}'), RDFS.label, Literal(f'{feat.capitalize()}={f.capitalize()}')))
                    g.add((URIRef(f'https://universaldependencies.org/la/feat/{feat.capitalize()}#{f.capitalize()}'), DC.title, Literal(f'{feat.capitalize()}#{f.capitalize()}')))
                    g.add((ud_node_uri, OA.hasBody, feat_node_uri))
    
            if token.form == 'Plato':
                g.add((powla_token_uri, RDFS.seeAlso, plato_uri))
            
            # Link to LiLa lemma
            try :
                for sent in lila_linking[row['id']]['sentences']:
                    for lila_token in sent:
                        if lila_token['token'] == token_form:
                            if len(lila_token['linking']) > 0:
                                lila_link_info = lila_token['linking'][0].split(":")
                                lila_lemma_id = lila_link_info[1]                
                                lila_lemma_uri = URIRef(f'http://lila-erc.eu/data/id/lemma/{lila_lemma_id}')  
                                g.add((lila_lemma_uri, RDF.type, LILA.lemma))
                                g.add((lila_lemma_uri, RDFS.label, Literal(token.lemma)))
                                g.add((lila_lemma_uri, DC.title, Literal(token.lemma)))
                                g.add((powla_token_uri, LILA.hasLemma, lila_lemma_uri))
                            else:
                                print(latin_expr + " "+ " unmatched token in LiLa " + token.form + " " + lila_token['token'])
            except :
                print(latin_expr + " "+ " error linking " + token.form)




a dicto secundum quid ad dictum simpliciter
a dicto simpliciter ad dictum secundum quid
a fortiori
a necesse ad esse valet consequentia
a posteriori
a priori
ab esse ad posse valet consequentia
ab ovo
actus purus
ad hoc
ad hoc hypothesis
ad infinitum
amicus Plato sed magis amica veritas
amor fati
anima mundi
argumentum ad baculum
argumentum ad hominem
argumentum ad populum
ars gratia artis
barbara
bellum omnium contra omnes
causa sine qua non
causa sui
ceteris paribus
characteristica universalis
cogito, ergo sum
conditio sine qua non
consequentia mirabilis
creatio ex nihilo
credo quia absurdum est
credo quia impossibile est
credo ut intelligam
cui bono?
de dicto
de facto
de gustibus non disputandum
de jure
de re
deus sive natura
dictum de omni et nullo
do ut des
dubito, ergo cogito, ergo sum
dum spiro, spero
ens causa sui
ens rationis
ens realissimum
entia non sunt multiplicanda praeter necessitatem
esse est percipi
ex falso quodlibet
ex nihilo nihil fit
ex post facto
ex vi terminorum


In [63]:
# Add links between concepts
for key, value in linked_concepts_dict.items():
     concept_uri = value['uri']
     for link in value['links']:
         g.add((concept_uri, SKOS.related, linked_concepts_dict[link]['uri']))

In [50]:
print(g.serialize(format="turtle"))

@prefix dc: <http://purl.org/dc/elements/1.1/> .
@prefix dct: <http://purl.org/dc/terms/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix lila: <http://lila-erc.eu/ontologies/lila/> .
@prefix lime: <http://www.w3.org/ns/lemon/lime#> .
@prefix lp: <http://latinphilosophy.org/> .
@prefix oa: <http://www.w3.org/ns/oa#> .
@prefix ontolex: <http://www.w3.org/ns/lemon/ontolex#> .
@prefix powla: <http://purl.org/powla/powla.owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix vartrans: <http://www.w3.org/ns/lemon/vartrans#> .

<http://latinphilosophy.org/deprel/expr-0/2-1> a powla:Relation ;
    rdfs:label "deprel: case" ;
    powla:hasLabel "case" ;
    powla:hasLayer <http://latinphilosophy.org/layer/dependency> ;
    powla:hasSource <http://latinphilosophy.org/token/expr-0/2> ;
    powla:hasTarget <http://latinphilosophy.org/token/expr-0/1> .

<http://latinphilosophy.org/deprel/expr-0/2-4> a powla:Relation ;
 

In [64]:
g.serialize(format="turtle", destination="Latin-Philosophical-Expression.ttl")

<Graph identifier=Nd199e7b850f341a08c04a9612ca26fd0 (<class 'rdflib.graph.Graph'>)>

In [57]:
 len(g)

19561